In [1]:
import glob
import os
import sys
import time
import random
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import librosa
import torch
import torch.nn as nn
import torchaudio.functional as F
import torchaudio.transforms as T
import torch.optim as optim
import torchaudio
import IPython.display as ipd

import pickle5 as pickle
import tqdm
import IPython.display

from itertools import chain
from tqdm import tqdm
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from tabulate import tabulate
from IPython.display import Audio

from collections import defaultdict

from torch.autograd import Variable
from torch.utils.data import Dataset
from scipy.io.wavfile import read

from linformer_pytorch import Linformer
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import ConcatDataset, DataLoader, Dataset
from torchvision import datasets, transforms
from functools import partial

from ray import tune
from ray.tune import JupyterNotebookReporter
from ray.tune.schedulers import ASHAScheduler
from ray.air import session
from ray.air.checkpoint import Checkpoint

from tqdm.notebook import tqdm

from torch.nn.utils.rnn import pad_sequence

In [2]:
random_seed = 17 # or any of your favorite number

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

np.random.seed(random_seed)

random.seed(random_seed)

In [3]:
!nvidia-smi

Sat Mar 25 10:22:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   37C    P8    N/A /  N/A |    240MiB /  4042MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [5]:
class SpeechCommandsDataset(Dataset):
    """Google Speech Commands dataset."""

    def __init__(self, root_dir, split, transform=None):
        """
        Args:
            root_dir (string): Directory with all the data files.
            split    (string): In ["train", "valid", "test"].
        """
        self.root_dir = root_dir
        self.split = split
        self.transform = transform

        self.number_of_classes = len(self.get_classes())

        self.class_to_file = defaultdict(list)

        self.valid_filenames = self.get_valid_filenames()
        self.test_filenames = self.get_test_filenames()

        for c in self.get_classes():
            file_name_list = sorted(os.listdir(self.root_dir + "dataset/" + c))
            for filename in file_name_list:
                if split == "train":
                    if (filename not in self.valid_filenames[c]) and (filename not in self.test_filenames[c]):
                        self.class_to_file[c].append(filename)
                elif split == "valid":
                    if filename in self.valid_filenames[c]:
                        self.class_to_file[c].append(filename)
                elif split == "test":
                    if filename in self.test_filenames[c]:
                        self.class_to_file[c].append(filename)
                else:
                    raise ValueError("Invalid split name.")

        self.filepath_list = list()
        self.label_list = list()
        for cc, c in enumerate(self.get_classes()):
            f_extension = sorted(list(self.class_to_file[c]))
            l_extension = [cc for i in f_extension]
            f_extension = [self.root_dir + "dataset/" + c + "/" + filename for filename in f_extension]
            self.filepath_list.extend(f_extension)
            self.label_list.extend(l_extension)
        self.number_of_samples = len(self.filepath_list)
    def __len__(self):
        return self.number_of_samples

    def __getitem__(self, idx): 
        sample = np.zeros((16000, ), dtype=np.float32)

        sample_file = self.filepath_list[idx]

        sample_from_file = read(sample_file)[1]

        sample[:sample_from_file.size] = sample_from_file
        sample = sample.reshape((16000, ))

        # It is better to stick to PyTorch functions
        # I swapped the `librosa´ mfcc resampling with this PyTorch alternative
        # however, the parameters are the same
        mfcc_transform = T.MFCC(
            sample_rate=16000,
            n_mfcc=20,
            melkwargs={
              'n_fft': 2048,
              'hop_length': 512})

        sample = torch.transpose(mfcc_transform(torch.tensor(sample)), 0, 1)

        #sample = librosa.feature.mfcc(y=sample, sr=16000, hop_length=512, n_fft=2048).transpose().astype(np.float32)

        # we apply the augmentations on the sample if it is defined
        if self.transform:
            sample = self.transform(samples=sample.reshape(32, 1, 20), sample_rate=16000)
            sample = sample.reshape(32, 20)

        label = self.label_list[idx]

        return sample, label
    

    def get_classes(self):
        return ["zero", "one", "two", "three", "four", "five", "six", "seven", "eight", "nine","right", 
                "left", "up", "down", "forward", "backward", "yes", "no", "stop", "start", "enable", 
                "disable", "ok", "cancel", "open", "close", "zoom in", "zoom out", "previous", "next", 
                "send", "receive", "move", "rotate", "record", "enter", "digit", "direction", "options", "undo"]

    def get_valid_filenames(self):
        class_names = self.get_classes()

        class_to_filename = defaultdict(set)
        with open(self.root_dir + "dataset/val.txt", "r") as fp:
            for line in fp:
                clean_line = line.strip().split("/")

                if clean_line[0] in class_names:
                    class_to_filename[clean_line[0]].add(clean_line[1])

        return class_to_filename

    def get_test_filenames(self):
        class_names = self.get_classes()

        class_to_filename = defaultdict(set)
        with open(self.root_dir + "dataset/test.txt", "r") as fp:
            for line in fp:
                clean_line = line.strip().split("/")

                if clean_line[0] in class_names:
                    class_to_filename[clean_line[0]].add(clean_line[1])

        return class_to_filename

In [6]:
from torch_audiomentations import Compose, Gain, PolarityInversion

# Initialize augmentation callable
train_transforms = Compose(
    transforms=[
        Gain(
            min_gain_in_db=-15.0,
            max_gain_in_db=5.0,
            p=0.5,
        ),
        PolarityInversion(p=0.5)
    ]
)
val_transforms = Compose(
    transforms=[
        Gain(
            min_gain_in_db=-15.0,
            max_gain_in_db=5.0,
            p=0.5,
        ),
        PolarityInversion(p=0.5)
    ]
)
test_transforms = Compose(
    transforms=[
        Gain(
            min_gain_in_db=-15.0,
            max_gain_in_db=5.0,
            p=0.5,
        ),
        PolarityInversion(p=0.5)
    ]
)

In [7]:
dataset_folder= os.path.join(os.getcwd(), "Arabic_Speech_command/")

train_dataset = SpeechCommandsDataset(dataset_folder, "train")

valid_dataset = SpeechCommandsDataset(dataset_folder, "valid")

test_dataset  = SpeechCommandsDataset(dataset_folder, "test")

### Augmented Datasets ###
# Note we should concatenate both the non-augmented and the augmented training datasets

train_dataset_aug = ConcatDataset([SpeechCommandsDataset(dataset_folder, "train", 
                                                         transform=train_transforms),
                                                         train_dataset])

valid_dataset_aug = SpeechCommandsDataset(dataset_folder, "valid", transform=val_transforms)

test_dataset_aug  = SpeechCommandsDataset(dataset_folder, "test", transform=test_transforms)

In [8]:
print(f"Training dataset, {len(train_dataset)} samples.")

print(f"Validation dataset, {len(valid_dataset)} samples.")

print(f"Testing dataset, {len(test_dataset)} samples.")

Training dataset, 7201 samples.
Validation dataset, 2399 samples.
Testing dataset, 2400 samples.


In [9]:
class NeuralNetworkModel(nn.Module):
    """Neural network model (Transformer-based).

    Args:
        idim (int): Input feature dimension.
        d_att (int): Attention dimension.
        n_heads (int): The number of attention heads.
        d_ff (int): Dimension of feed forward network.
        dropout_rate (float): Dropout rate.
        n_layers (int): The number of encoder layers.
        d_linear (int): Dimension of a hidden layer of the classifier.
        n_classes (int): The number of the output classes.

    """

    def __init__(
        self,
        idim=13,
        d_att=64,
        n_heads=4,
        d_ff=512,
        dropout_rate=0.1,
        n_layers=3,
        n_classes=40
    ):
        super().__init__()
        self.subsampling = Subsampling(idim=idim, d_att=d_att)
        self.positional_encoding = PositionalEncoding()
        self.encoder_layers = nn.Sequential()
        for i in range(n_layers):
            self.encoder_layers.add_module(
                f'EncoderLayer{i}', 
                TransformerEncoderLayer(d_att, n_heads, d_ff, dropout_rate)
            )
        self.norm = nn.LayerNorm(d_att)
        self.dropout = nn.Dropout(dropout_rate)
        self.out = nn.Linear(d_att, n_classes)

    def forward(self, x):
        """Recognize the input speech commands.

        Args:
            x (torch.Tensor): Input features (batch, tmax, idim).

        Returns:
            torch.Tensor: Recognized classes (batch, num_classes).

        """   
        # Transformer encoder
        x = self.subsampling(x)
        x = self.positional_encoding(x)
        x = self.encoder_layers(x)
        x = self.norm(x)
        # Classifier
        x = torch.mean(x, dim=1)  # (b, t, d_att) -> (b, d_att)
        x = self.dropout(x)
        x = self.out(x)

        return x

class TransformerEncoderLayer(nn.Module):
    """A Transformer encoder layer.

    Args:
        d_att (int): Attention dimension.
        d_head (int): The number of attention heads.
        d_ff (int): Dimension of feed forward network.
        dropout_rate (float): Dropout rate.

    """

    def __init__(self, d_att, d_head, d_ff, dropout_rate=0.1):
        super().__init__()
        self.mha = MultiHeadAttention(d_att, d_head, dropout_rate)
        self.ff = FeedForward(d_att, d_ff, dropout_rate)
        self.norm_mha = nn.LayerNorm(d_att)
        self.norm_ff = nn.LayerNorm(d_att)
        self.dropout = nn.Dropout(dropout_rate)
    
    def forward(self, x):
        """

        Args:
            x (torch.Tensor): Pre-encoded inputs (batch, tmax, d_att).

        Returns:
            torch.Tensor: Encoded outputs (batch, tmax, d_att).

        """
        # Multi-Head Attention
        res = x
        x = self.norm_mha(x)
        x = res + self.dropout(self.mha(x, x, x))

        # Feed-Forward
        res = x
        x = self.norm_ff(x)
        x = res + self.dropout(self.ff(x))

        return x
class Subsampling(nn.Module):
    """Convolutional Subsampling.

    Args:
        idim (int): Input feature dimension.
        d_att (int): Attention dimension.

    """

    def __init__(self, idim, d_att):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, d_att, kernel_size=(3, 3), stride=(2, 2)),
            nn.ReLU(),
            nn.Conv2d(d_att, d_att, kernel_size=(3, 3), stride=(2, 2)),
            nn.ReLU(),
        )
        self.linear = nn.Linear(256, d_att)

    def forward(self, x):
        """

        Args:
            x (torch.Tensor): Input features (batch, tmax, idim).

        Returns:
            torch.Tensor: Subsampled features (batch, tmax', d_att).

        """
        x = x.unsqueeze(1)  # (b, t, idim) -> (b, c=1, t, idim)
        x = self.conv(x)
        b, c, t, f = x.size()
        #x = x.view(x.size(0), -1)
        x = x.transpose(1, 2).contiguous().view(b, t, c * f)  # (b, c, t, f) -> (b, t, c * t)
        
        x = self.linear(x)

        return x

class PositionalEncoding(nn.Module):
    """Positional Encoding.

    Args:
        idim (int): Input feature dimension.

    """

    def __init__(self):
           super().__init__()
        
    def forward(self, x):
        """

        Args:
            x: (torch.Tensor): Subsampled features (batch, tmax, d_att).

        Returns:
            torch.Tensor: Encoded features (batch, tmax, d_att).

        """
        _, tmax, d_att = x.size()
        pos = torch.arange(0, tmax, dtype=torch.float32).unsqueeze(1)
        pe = torch.zeros(1, tmax, d_att, dtype=torch.float32).to(x.device)
        pe[:, :, 0::2] = torch.sin(pos / torch.pow(10000, torch.arange(0, d_att, 2) / d_att))
        pe[:, :, 1::2] = torch.cos(pos / torch.pow(10000, torch.arange(0, d_att, 2) / d_att))
        x = x + pe

        return x

class MultiHeadAttention(nn.Module):
    """Multi-Head Attention.

    Args:
        d_att (int): Dimension of attention.
        d_head (int): The number of attention heads.
        dropout_rate (float): Dropout rate.

    """

    def __init__(self, d_att, n_heads, dropout_rate):
        super().__init__()
        self.linear_q = nn.Linear(d_att, d_att)
        self.linear_k = nn.Linear(d_att, d_att)
        self.linear_v = nn.Linear(d_att, d_att)
        self.linear_head = nn.Linear(d_att, d_att)
        self.dropout = nn.Dropout(dropout_rate)
        self.n_heads = n_heads
        self.d_comn = d_att // self.n_heads

    def forward(self, q, k, v):
        """

        Args:
            q: (torch.Tensor): Query (batch, tmax, d_att).
            k: (torch.Tensor): Key (batch, tmax, d_att).
            v: (torch.Tensor): Value (batch, tmax, d_att).
            Returns:
            torch.Tensor: Output shape (batch, tmax, d_att).

        """
        # Linear
        qw = self.linear_q(q)
        kw = self.linear_k(k)
        vw = self.linear_v(v)

        # Reshape tensor (b, t, d_att) -> (b, n_heads, t, d_comn)
        b, t, d_att = q.size()
        qw = qw.view(b, t, self.n_heads, self.d_comn).transpose(1, 2)
        kw = kw.view(b, t, self.n_heads, self.d_comn).transpose(1, 2)
        vw = vw.view(b, t, self.n_heads, self.d_comn).transpose(1, 2)

        # Dot-attention
        matmul = torch.matmul(qw, kw.transpose(2, 3)) 
        scale = matmul / torch.sqrt(torch.tensor(self.d_comn))
        softmax = torch.softmax(scale, dim=-1)
        att = torch.matmul(self.dropout(softmax), vw)  # (b, n_heads, t, d_comn)

        # Concatenate
        att = att.transpose(1, 2).contiguous().view(b, -1, self.n_heads * self.d_comn)  # (b, t, d_att)

        # Linear
        mha = self.linear_head(att)

        return mha

class FeedForward(nn.Module):
    """Feed-Forward Network.

    Args:
        d_ff (int): Dimension of feed-forward network.

    """

    def __init__(self, d_att, d_ff, dropout_rate):
        super().__init__()
        self.ff = nn.Sequential(
            nn.Linear(d_att, d_ff), nn.ReLU(), nn.Dropout(dropout_rate), nn.Linear(d_ff, d_att)
        )

    def forward(self, x):
        """

        Args:
            x: (torch.Tensor): Input shape (batch, tmax, d_att).

        Returns:
            torch.Tensor: Output shape (batch, tmax, d_att).

        """
        return self.ff(x)

In [10]:
# define some metric lists
def train(config, 
          train_set, 
          val_set, 
          results_dir='./results',
          continue_training=False,
          epochs=5, 
          plot=False):
    
    model = NeuralNetworkModel(dropout_rate=config["dropout_rate"], 
                               n_heads=config["n_heads"]).to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config["lr"])
    
    if continue_training:
        model_state, optimizer_state = torch.load(
            os.path.join(results_dir, "checkpoint"))
        model.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)
    
    train_loader = DataLoader(
        train_set,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=8)
    
    valid_loader = DataLoader(
        val_set,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=8)
    
    #store evaluation metrics in these lists
    train_acc_list = []
    train_loss_list = []
    dev_acc_list = []
    dev_loss_list = []
    
    # best loss tracking for model checkpointing
    best_loss = float("inf")

    # Train the model
    for epoch in range(epochs):

        batch_train_loss = 0
        batch_train_acc = 0
        batch_dev_loss = 0
        batch_dev_acc = 0

        # Training
        model.train()
        for feats, labels in train_loader:
            feats, labels = feats.to(device), labels.to(device)

            # Reset gradients
            optimizer.zero_grad()

            # Forward
            outputs = model(feats)
            loss = criterion(outputs, labels)

            # Backward
            loss.backward()

            # Update weights
            optimizer.step()

            batch_train_loss += loss.item()
            batch_train_acc += (outputs.max(1)[1] == labels).sum().item()

        train_loss_list.append(batch_train_loss / len(train_loader))
        train_acc_list.append(batch_train_acc / len(train_loader.dataset))
        # Validation
        
        model.eval()
        with torch.no_grad():
            for feats, labels in valid_loader:
                feats, labels = feats.to(device), labels.to(device)

                # Forward
                outputs = model(feats)
                loss = criterion(outputs, labels)

                batch_dev_loss += loss.item()
                batch_dev_acc += (outputs.max(1)[1] == labels).sum().item()

        dev_loss_list.append(batch_dev_loss / len(valid_loader))
        dev_acc_list.append(batch_dev_acc / len(valid_loader.dataset))
        
        os.makedirs(results_dir, exist_ok=True)
        torch.save((model.state_dict(), optimizer.state_dict()), os.path.join(results_dir, f"checkpoint{epoch}.pt"))
        
        checkpoint = Checkpoint.from_directory(results_dir)
        session.report({
                        "loss" : dev_loss_list[-1],
                        "accuracy" : dev_acc_list[-1]
                        },
                        checkpoint=checkpoint)
        
        # plot results if flag is True
        if plot:
            fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 6.5))

            ax[0].plot(range(len(train_loss_list)), train_loss_list, label='train loss')
            ax[0].plot(range(len(dev_loss_list)), dev_loss_list, label='val loss')
            ax[0].set_title('Loss Evolution')

            ax[1].plot(range(len(train_acc_list)), train_acc_list, label='train accuracy')
            ax[1].plot(range(len(dev_acc_list)), dev_acc_list, label='val accuracy')
            ax[1].set_title('Accuracy Evolution')

            ax[0].legend()
            ax[1].legend()

            plt.show()
        
def test(model, test_set, device, batch_size=64):
    best_loss = float("inf")
    
    test_loader = DataLoader(
        test_set, batch_size=batch_size, shuffle=False, num_workers=4)

    # Test
    test_start_time = time.time()
    
    batch_test_loss = 0
    batch_test_acc = 0

    with torch.no_grad():
        for feats, labels in test_loader:
            feats, labels = feats.to(device), labels.to(device)

            # Forward
            outputs = model(feats)
            batch_test_acc += (outputs.max(1)[1] == labels).sum().item()

    test_mean_acc = batch_test_acc / len(test_loader.dataset)

    # Print out test time
    test_time = time.time() - test_start_time
    print(f">> Test Time: {test_time}")
    
    return test_mean_acc

# Train/Validation Runs

In [ ]:
num_samples=500
max_num_epochs=50

config = {
    "dropout_rate": tune.loguniform(1e-1, 1.5e-1),
    "n_heads": tune.choice([2, 4, 8, 16, 32]),
    "lr": tune.loguniform(1e-4, 1e-2),
    "batch_size": tune.choice([32, 64, 128])
}

scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)

reporter = JupyterNotebookReporter(
        overwrite=True,
        metric_columns=["loss", "accuracy", "training_iteration"])

result = tune.run(
            partial(train, 
                    train_set=train_dataset_aug, 
                    val_set=valid_dataset_aug,
                    epochs=max_num_epochs),
            resources_per_trial={"cpu": 8, "gpu": 1},
            config=config,
            num_samples=num_samples,
            scheduler=scheduler,
            progress_reporter=reporter)

best_trial = result.get_best_trial("loss", "min", "last")
print(f"Best trial config: {best_trial.config}")
print(f"Best trial final validation loss: {best_trial.last_result['loss']}")
print(f"Best trial final validation accuracy: {best_trial.last_result['accuracy']}")

2023-03-25 10:23:06,367	INFO worker.py:1550 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Trial name,accuracy,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,loss,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_a8179_00000,0.737807,2023-03-25_10-37-18,True,,cc7629795ca94f94868caa0a82b689e5,mbe19,50,1.03222,139.174.120.12,30338,True,846.637,17.1035,846.637,1679737038,0,,50,a8179_00000,0.00304747
train_a8179_00001,0.692789,2023-03-25_10-46-37,True,,690dcca6431b4e3aa71333d777ea9d09,mbe19,32,1.51917,139.174.120.12,11423,True,553.659,17.412,553.659,1679737597,0,,32,a8179_00001,0.00311875
train_a8179_00002,0.0304293,2023-03-25_10-47-01,True,,fc9b4f1aa1594028982d9bb5e864a64a,mbe19,1,3.68595,139.174.120.12,19554,True,19.3095,19.3095,19.3095,1679737621,0,,1,a8179_00002,0.00309086
train_a8179_00003,0.731555,2023-03-25_11-01-25,True,,8f9366132bce428b8309602be2466b14,mbe19,50,1.52538,139.174.120.12,19872,True,860.118,17.2755,860.118,1679738485,0,,50,a8179_00003,0.00294733
train_a8179_00004,0.695707,2023-03-25_11-06-13,True,,5628ef452928423983619afb6bd78fd5,mbe19,16,1.35881,139.174.120.12,732,True,282.774,17.8081,282.774,1679738773,0,,16,a8179_00004,0.00345778
train_a8179_00005,0.0637766,2023-03-25_11-06-36,True,,5ee0b78d29694a4baa14fa2cb44e5229,mbe19,1,3.5599,139.174.120.12,5491,True,19.5659,19.5659,19.5659,1679738796,0,,1,a8179_00005,0.00290728
train_a8179_00006,0.0516882,2023-03-25_11-07-00,True,,a759e89120ff4ba2a1abac0bb75e85de,mbe19,1,3.59668,139.174.120.12,5920,True,19.0217,19.0217,19.0217,1679738820,0,,1,a8179_00006,0.00307059
train_a8179_00007,0.352647,2023-03-25_11-07-41,True,,7b031f1138f3498f9626ae5a47ae8c2f,mbe19,2,2.25204,139.174.120.12,6303,True,37.3963,17.7016,37.3963,1679738861,0,,2,a8179_00007,0.00297427
train_a8179_00008,0.356398,2023-03-25_11-08-20,True,,459205b3c1f046659a1c4acb189fc59e,mbe19,2,2.31912,139.174.120.12,6882,True,35.7536,16.7935,35.7536,1679738900,0,,2,a8179_00008,0.00290608
train_a8179_00009,0.200083,2023-03-25_11-08-44,False,,42f018b4f9ae452187bbee392a6541ce,mbe19,1,2.90575,139.174.120.12,7481,True,19.763,19.763,19.763,1679738924,0,,1,a8179_00009,0.0028522


In [ ]:
best_trial = result.get_best_trial("loss", "min", "last")

best_trained_model = NeuralNetworkModel(dropout_rate=best_trial.config["dropout_rate"], 
                                        n_heads=best_trial.config["n_heads"])

best_trained_model.to(device)

best_checkpoint = result.get_best_checkpoint(trial=best_trial, metric="loss", mode="min")
best_checkpoint_dir = best_checkpoint.to_directory(path="results")

model_state, optimizer_state = torch.load(os.path.join(best_checkpoint_dir, "checkpoint.pt"))
best_trained_model.load_state_dict(model_state)

test_acc = test(model=best_trained_model, test_set=test_dataset,
                device=device, batch_size=best_trial.config["batch_size"])

print(f"Best trial test set accuracy: {test_acc}")

In [ ]:
## Train Data (augmented) with Validation Data (Augmented)
# We can notice that the max accuracy is reaching a value of ~ 77%

# Build a model
model = NeuralNetworkModel(dropout_rate=best_trial.config["dropout_rate"], 
                               n_heads=best_trial.config["n_heads"]).to(device)

#best_trial.config["lr"]
#best_trial.config["batch_size"]
        
print(f"# model parameters: {sum(p.numel() for p in aug_model.parameters()):,}")

# Define an optimizer and a loss function
optimizer = optim.Adam(aug_model.parameters(), lr=best_trial.config["lr"])
loss_fn = nn.CrossEntropyLoss()

epochs = 100

train(, optimizer=optimizer, 
      train_loader=train_loader_aug, valid_loader=valid_loader_aug, 
      criterion=aug_loss_fn, epochs=epochs, plot=True)

# Test Runs

In [ ]:
## Testing the model with the test augmented data

test(model=aug_model, test_loader=test_loader_aug, criterion=aug_loss_fn)

In [ ]:
## Testing the model with the test augmented data

test(model=baseline_model, test_loader=test_loader_aug, criterion=baseline_loss_fn)

In [ ]:
## Testing the model with the test non-augmented data

test(model=aug_model, test_loader=test_loader, criterion=aug_loss_fn)

In [ ]:
## Testing the model with the test non-augmented data

test(model=baseline_model, test_loader=test_loader, criterion=baseline_loss_fn)